# Install spleeter

In [ ]:
!apt install ffmpeg
!apt install imagemagick
!pip install virtualenv
!virtualenv spleeter-env
!virtualenv whisper-env


Install Spleeter in other venv

In [4]:
!source spleeter-env


/bin/bash: line 1: source: spleeter-env: is a directory


In [ ]:
pip install spleeter

Install Wisper in other Venv.

In [1]:
from IPython.display import Audio

# Separate from command line

In [2]:
# @title 🌴 Change the values in this section

## @markdown Select the source of the audio/video file to be transcribed
#input_format = "youtube" #@param ["youtube", "gdrive", "local"]

# @markdown Enter the URL of the YouTube video or the path of the audio file to be transcribed
file = "https://archive.org/download/caturnugra_gmail_Rock/01%20-%20I%20Want%20It%20That%20Way.mp3" #@param {type:"string"}
!wget -O 'song.mp3' $file

--2025-05-21 08:22:36--  https://archive.org/download/caturnugra_gmail_Rock/01%20-%20I%20Want%20It%20That%20Way.mp3
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia800604.us.archive.org/12/items/caturnugra_gmail_Rock/01%20-%20I%20Want%20It%20That%20Way.mp3 [following]
--2025-05-21 08:22:36--  https://ia800604.us.archive.org/12/items/caturnugra_gmail_Rock/01%20-%20I%20Want%20It%20That%20Way.mp3
Resolving ia800604.us.archive.org (ia800604.us.archive.org)... 207.241.230.64
Connecting to ia800604.us.archive.org (ia800604.us.archive.org)|207.241.230.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8747214 (8.3M) [audio/mpeg]
Saving to: ‘song.mp3’

song.mp3            100%[===================>]   8.34M  48.1MB/s    in 0.2s    

2025-05-21 08:22:36 (48.1 MB/s) - ‘song.mp3’ saved [8747214/8747214]



In [ ]:
Audio('song.mp3')

In [ ]:
!spleeter separate --help

In [ ]:
!spleeter separate -o output/ 'song.mp3'

INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 2stems archive
INFO:spleeter:2stems model file(s) extracted


In [ ]:
!ls output/audio_example

In [ ]:
Audio('output/song/vocals.wav')
Audio('output/song/accompaniment.wav')

In [ ]:
!source whisper-env
# Fix for whisper + torch DispatchKey.Meta bug in Colab
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2
!pip install -U openai-whisper


In [ ]:
import whisper
import json

model = whisper.load_model("base")
result = model.transcribe("output/song/vocals.wav")

with open("output/song/segments.json", "w", encoding="utf-8") as f:
    json.dump(result["segments"], f, indent=2)

Convert Song to Karaoke Video

In [ ]:
from moviepy.editor import AudioFileClip, ImageClip, CompositeVideoClip
from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import ColorClip
import os, json

FONT_PATH = "C:/Windows/Fonts/arial.ttf"  # adjust for your OS
FONT_SIZE = 48
SCREEN_SIZE = (1280, 720)
LINE_HEIGHT = 60
FPS = 24

def make_line_image(text, size=SCREEN_SIZE, highlight=False):
    img = Image.new("RGBA", size, (0, 0, 0, 255))
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(FONT_PATH, FONT_SIZE)

    color = (255, 255, 0, 255) if highlight else (255, 255, 255, 255)
    bbox = draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    x = (size[0] - text_width) // 2
    y = size[1] - 150
    draw.text((x, y), text, font=font, fill=color)

    return img

def main():
    with open("segments.json", "r", encoding="utf-8") as f:
        segments = json.load(f)

    audio = AudioFileClip("instrumental.wav")
    #background = ImageClip("bg.jpg").set_duration(audio.duration).resize(SCREEN_SIZE)
    background = ColorClip(size=SCREEN_SIZE, color=(0, 0, 0)).set_duration(audio.duration)

    clips = []
    for i, seg in enumerate(segments):
        img = make_line_image(seg["text"], highlight=True)
        fname = f"_line_{i}.png"
        img.save(fname)

        clip = ImageClip(fname).set_start(seg["start"]).set_duration(seg["end"] - seg["start"]).fadein(0.3).fadeout(0.3)
        clips.append(clip)

    final = CompositeVideoClip([background] + clips).set_audio(audio).set_duration(audio.duration)
    final.write_videofile("karaoke_fast.mp4", fps=FPS)

    for f in os.listdir():
        if f.startswith("_line_") and f.endswith(".png"):
            os.remove(f)

if __name__ == "__main__":
    main()
